In [2]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from data_cleaning.data_cleaning import get_zimmer, get_stockwerk, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_kaution, get_stadtmitte, get_kueche, get_bad, get_mobliert, get_quadratmeter

/home/sven/dev/DHBW-KI-Laborarbeit/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file_path = '../../data/Wohnungen_2.csv'
data = pd.read_csv(file_path, sep=';')

bewohnerkategorie = ["Studierende", "Kleinfamilie", "DINK", "SingleHighIncome", "Expatriate", "Rentnerpaar"]
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else row.idxmax(), axis=1)

data = data[data['Bewohnerkategorie'] != 'Keine']

columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Entfernung zur Stadtmitte'] = data['Entfernung zur Stadtmitte'].apply(get_stadtmitte)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

#Code Cleaning

    Zimmerzahl  Stockwerk  Hausmeister  Kindergarten  Schule  S-Bahn  Garage  \
3            0          3            1             2       0       0       0   
4            2          4            0             1       2       0       0   
6            2          1            0             2       2       3       0   
7            2          0            0             2       2       3       1   
16           0          4            0             0       2       2       1   

    Miete  Nebenkosten  Alter  ...  Kehrwoche  Moebliert  Quadratmeter  \
3     276           50     63  ...          0          0            25   
4     626          226     18  ...          0          0            96   
6     626          226      6  ...          0          0            96   
7     726          226      0  ...          0          0           106   
16    276           75     26  ...          1          0            25   

    Studierende  Kleinfamilie  DINK  SingleHighIncome  Expatriate  \
3    

/tmp/ipykernel_7248/3487156503.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [4]:
#Data Processing and Training
model = BayesianNetwork([
    ('Zimmerzahl', 'Bewohnerkategorie'),
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Hausmeister', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('Schule', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie'),
    ('Garage', 'Bewohnerkategorie'),
    ('Miete', 'Bewohnerkategorie'),
    ('Nebenkosten', 'Bewohnerkategorie'),
    ('Alter', 'Bewohnerkategorie'),
    ('Aufzug', 'Bewohnerkategorie'),
    ('Entfernung zur Stadtmitte', 'Bewohnerkategorie'),
    ('Kaution', 'Bewohnerkategorie'),
    ('Kueche', 'Bewohnerkategorie'),
    ('Bad', 'Bewohnerkategorie'),
    ('Balkon', 'Bewohnerkategorie'),
    ('Terrasse', 'Bewohnerkategorie'),
    ('Kehrwoche', 'Bewohnerkategorie'),
    ('Moebliert', 'Bewohnerkategorie'),
    ('Quadratmeter', 'Bewohnerkategorie')
])

# Training des Bayes-Netzes
model.fit(data, estimator=MaximumLikelihoodEstimator)

In [10]:
# Inferenz durchführen
inference = VariableElimination(model)

# Beispiel-Eingabe: Teilweise bekannte Daten
example_input = {
    "Balkon": 1,
    "Terrasse": 2,
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.map_query(variables=["Bewohnerkategorie"], evidence=example_input)


print("Klassifikationsergebnis:")
print(prediction)

Eliminating: Schule: 100%|██████████| 3/3 [00:00<00:00, 3694.34it/s]

Klassifikationsergebnis:
{'Bewohnerkategorie': 'Kleinfamilie'}
